## Matching colour histograms of masked image textures

The code is based on the blogpost [Histogram matching with opencv](https://pyimagesearch.com/2021/02/08/histogram-matching-with-opencv-scikit-image-and-python/) and presents an open source alternative to Peter Falkingham's blogpost on [Colour matching textures of two models with photoshop](https://peterfalkingham.com/2021/10/02/colour-matching-textures/), extending his masking method to automatically [exclude black background elements](https://gist.github.com/tayden/dcc83424ce55bfb970f60db3d4ddad18) resulting from texture islands from the histogram matching process.

![image](texture_histogram_matching.jpg)

In [ ]:
from skimage import io, exposure
import os
import numpy as np

Replcae the locations of your **source** and **reference** images according to where they are located on your drive. For batch processing of matching an entire **folder of images** to a **refrence** image, simply set **BATCH** to **True** and provide the path of to the folder containing your source images as the **source**

In [ ]:
source = "input.png"
reference = "reference.png"
BATCH = False

In [ ]:
# Histogram matching with masked image
def match_histograms(image, reference, image_mask, reference_mask, fill_value=0):
    # adapted from https://gist.github.com/tayden/dcc83424ce55bfb970f60db3d4ddad18
    # to include masks for the input and reference image
    masked_source_image = np.ma.array(image, mask=image_mask)
    masked_reference_image = np.ma.array(reference, mask=reference_mask)
    
    matched = np.ma.array(np.empty(image.shape, dtype=image.dtype),
                          mask=image_mask, fill_value=fill_value)
    
    for channel in range(masked_source_image.shape[-1]):
        matched_channel = exposure.match_histograms(masked_source_image[...,channel].compressed(), 
                                                    masked_reference_image[...,channel].compressed())
        
        # Re-insert masked background
        mask_ch = image_mask[...,channel]
        matched[..., channel][~mask_ch] = matched_channel.ravel() 
    
    return matched.filled()

In [ ]:
if BATCH:
    src = []
    for filename in os.listdir(source):
        f = os.path.join(source, filename)
        if os.path.isfile(f):
            src.append(f)
            
else:
    src = [source]

for src_file in src:
    print("[INFO] processing ", src_file)
    src = io.imread(src_file)
    ref = io.imread(reference)

    # Get a mask that matches image.shape, with mask being where pixel val across channels is 0
    src_mask = np.repeat(np.expand_dims(np.all(src == 0, axis=2), axis=2), repeats=3, axis=2)
    ref_mask = np.repeat(np.expand_dims(np.all(ref == 0, axis=2), axis=2), repeats=3, axis=2)

    print("[INFO] performing histogram matching...")

    # Do the masked histogram matching
    output = match_histograms(src, ref, src_mask, ref_mask, fill_value=0)

    io.imshow(output)
    io.imsave(src_file[:-4] + "_histogram_matched.png",output.astype("uint8"))

    print("[INFO] Output written to source folder...")